In [ ]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import numpy as np

"""
Update Parameters Here
"""
COLLECTION_NAME = "quaks"
CONTRACT = '0x07bbdaf30e89ea3ecf6cadc80d6e7c4b0843c729'
METHOD = "raritytools"

# Need to updte these depending on the collection
START_TIME = datetime.datetime.strptime("2021-08-24T21:18:00", "%Y-%m-%dT%H:%M:%S")
LAST_MINT = datetime.datetime.strptime("2021-09-01T04:38:00", "%Y-%m-%dT%H:%M:%S")
TOP_N = 1000
TIME_DELTA = 1 # length of a single mint window

# Need to update this with list of addresses that you saw with hits
hits = []

In [ ]:
"""
Generate Plot
"""
RARITY_CSV = "../metadata/rarity_data/{}_{}.csv".format(COLLECTION_NAME, METHOD)
RARITY_DB = pd.read_csv(RARITY_CSV)
MINT_DB = pd.read_csv("../minting_data/{}_minting.csv".format(COLLECTION_NAME))

MINT_DB = MINT_DB[MINT_DB['to_account'].isin(hits)]


MINT_DB['time'] =  pd.to_datetime(MINT_DB['time'], format="%Y-%m-%dT%H:%M:%S")
MINT_DB = MINT_DB.sort_values(by=['time'])


time_diff = (LAST_MINT - START_TIME)
total_seconds = time_diff.total_seconds()
minutes = total_seconds/60
iterations = int(minutes / TIME_DELTA)
print("Number of time blocks: ", iterations)
current_time = START_TIME
count = 0


dict_list = []
for i in range(0, iterations):
    new_time = current_time + datetime.timedelta(minutes = TIME_DELTA)
   
    sliding_window = MINT_DB[(MINT_DB['time'] < new_time) & (MINT_DB['time'] > current_time)]

    rarest_token_rank = sliding_window['rank'].min()

    data = {
        'starting_time' : current_time,
        'ending_time'   : new_time,
        'mints' : len(sliding_window),
        'lowest_rank_in_window' : rarest_token_rank
    }
    
    dict_list.append(data)
    count += data['mints']
    current_time = new_time

volume_data = pd.DataFrame(dict_list)

volume_data.to_csv("volume_spikes/{}_volume.csv".format(COLLECTION_NAME))

fig, ax = plt.subplots(1,1, figsize=(14, 7))
plt.bar(volume_data['starting_time'], volume_data['mints'], width=0.01)
plt.xlabel('time', fontsize=12)
plt.ylabel('minted in window', fontsize=12)
ax.set_xticks(ax.get_xticks()[::2])